In [1]:
# Fusion ART Model for Professor Data using Word2Vec
# Python fusionART.py code by Dr. Budhitama Subagdja, modified for this dataset by Patrick Tjahjadi

# ----- Imported Libraries -----
from fusionART import *

from collections import defaultdict
from collections import Counter

import pandas as pd

import math

import random

from numpy import array
from numpy import argmax

from itertools import islice

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk import ngrams
from nltk.corpus import stopwords

import gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import gensim.downloader as api
from gensim import corpora

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import csv

import wikipedia

import dill
# -------------------------------

data = pd.read_csv('SCSE ProfProfile.csv', encoding='ISO-8859-1')

In [2]:
# Load a session
dill.load_session('notebook_env_wikipedia.db')

In [2]:
# Function to convert 1D lists into 2D, based on the number of elements per list.
def convert_to_2d(list_to_convert, list_per_entry): 
    it = iter(list_to_convert) 
    return [list(islice(it, i)) for i in list_per_entry] 

In [3]:
# Function to clean words from punctuation and remove capital case to standardise text tokens
def clean_text(word):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_123456789~'''
    no_punct = ""
    for char in word:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct.lower()

In [4]:
# Function accepts a vector and normalise its values to a range between 0 and 1
def normalise_vector(vector):
    normalised_vector = [0] * len(vector)
    max_value = max(vector)
    min_value = min(vector)
    for index in range(0, len(vector)):
        normalised_vector[index] = (vector[index] - min_value) / (max_value - min_value)
    return normalised_vector

In [134]:
# Function to search for n-grams for n up to 6 and return the n-grammed text.
# Functions accepts list of n-grams to be formed, the text and the common n-grams within the text, respectively.
def search_for_ngrams(ngram_list, text_list):
    ngrammed_text = []
    for i in range(0, len(text_list)):
        bigram = text_list[i-1].lower()+"_"+text_list[i].lower()
        trigram = text_list[i-2].lower()+"_"+text_list[i-1]+"_"+text_list[i].lower()
        fourgram = text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        fivegram = text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        sixgram = text_list[i-5].lower()+"_"+text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        if bigram in ngram_list:
            ngrammed_text.append(bigram)
        if trigram in ngram_list:
            ngrammed_text.append(trigram)
        if fourgram in ngram_list:
            ngrammed_text.append(fourgram)
        if fivegram in ngram_list:
            ngrammed_text.append(fourgram)
        if sixgram in ngram_list:
            ngrammed_text.append(fourgram)
        ngrammed_text.append(text_list[i])
    return ngrammed_text


In [6]:
# Convert each attribute of the dataset to separate lists
name = []
group = []
university = []
research_interest = []
for idx, row in data.iterrows():
    name.append(row[0])
    group.append(row[1])
    university.append(row[2])
    research_interest.append(row[3])

# Problem: research_interest and university uses semicolons to denote multiple interests and universities
# Change their format to a list of universities and research interests of a person

university_preprocessed = []
for entry in university:
    uni = entry.split(";")
    university_preprocessed.append(uni)

research_interest_preprocessed = []
for entry in research_interest:
    interests = entry.split(";")
    research_interest_preprocessed.append(interests)    

    
# Break university_processed and research_interest_processed to 1D lists for one hot encoding
university_1d = []
for universities in university_preprocessed:
    for uni in universities:
        university_1d.append(uni)
        
research_interest_1d = []
for interests in research_interest_preprocessed:
    for interest in interests:
        research_interest_1d.append(interest)
        

In [7]:
# Determine the number of universities and research interests per professor

universities_per_person = []
for universities in university_preprocessed:
    universities_per_person.append(len(universities))
    

research_interest_per_person = []
for research_interests in research_interest_preprocessed:
    research_interest_per_person.append(len(research_interests))
    

In [8]:
# Research interest preprocessing, to consider each research keywords as a phrase
for index in range(0, len(research_interest_1d)):
    research_interest_1d[index] = research_interest_1d[index].replace(" ", "_")
    research_interest_1d[index] = research_interest_1d[index].lower()
    research_interest_1d[index] = research_interest_1d[index].replace(":", "")
    research_interest_1d[index] = research_interest_1d[index].replace(",", "")


In [9]:
# Create a list of unique names, groups, universities and research interests to define the model schema
# (I refrained from using sets because it would randomise the one-hot ordering)
unique_name = []
unique_group = []
unique_university = []
unique_research_interest = []
[unique_name.append(i) for i in name if i not in unique_name]
[unique_group.append(i) for i in group if i not in unique_group]
[unique_university.append(i) for i in university_1d if i not in unique_university]
[unique_research_interest.append(i) for i in research_interest_1d if i not in unique_research_interest]
pass


In [83]:
# Gather text tokens from Wikipedia articles for all research keywords 
wiki_text = []
for keyword in research_interest_1d:
    try:
        p = wikipedia.page(keyword)
        wiki_text.append(p.content.split())
        
    # Include exception handling if the Wikipedia article can not be found
    except wikipedia.exceptions.PageError:
        pass
    except wikipedia.DisambiguationError:
        pass



C:\Users\patpa\Anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\patpa\Anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [164]:
# Remove common stop words from the retrieved Wikipedia articles
eng_stopwords = set(stopwords.words('english'))
for text in wiki_text:
    for word in text:
        if word in eng_stopwords:
            text.remove(word)

In [182]:
# Search for research keyword n-grams from Wikipedia articles and find most common n-grams

trigger = 0
for text in wiki_text:
    string = " ".join(text)
    if (trigger == 0):
        bigram_counts = Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts = Counter(ngrams(string.split(), 3)).most_common(10)
        trigger += 1
    else:
        bigram_counts += Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts += Counter(ngrams(string.split(), 3)).most_common(10)




In [183]:
# Create a list of common bigrams and trigrams from the Wikipedia articles
common_ngrams = bigram_counts + trigram_counts
common_ngram_list = []
for each_ngram in common_ngrams:
    common_ngram_list.append("_".join(each_ngram[0]))


In [184]:
# Search for research keyword n-grams from the Wikipedia articles and clean them
ngram_wiki_tokens = []

for text in wiki_text:
    cleaned_text = []
    for word in text:
        cleaned_text.append(clean_text(word))
    ngram_wiki_tokens.append(search_for_ngrams(unique_research_interest + common_ngram_list, cleaned_text))

In [186]:
# Open the custom corpus text file and tokenise them. Group them with 2500 words each
testtext = []
with open('it_text.txt','r', encoding="utf-8") as f:
    for line in f:
        for word in line.split():
            testtext.append(word)  
            

# Remove stopwords in the custom text corpus
for word in testtext:
    if word in eng_stopwords:
        testtext.remove(word)
        
test_tokens = convert_to_2d(testtext, [2500] * (math.ceil(len(testtext)/2500)))


In [187]:
# Search for research keyword n-grams from Wikipedia articles and find most common n-grams

trigger = 0
for text in test_tokens:
    string = " ".join(text)
    if (trigger == 0):
        bigram_counts = Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts = Counter(ngrams(string.split(), 3)).most_common(10)
        trigger += 1
    else:
        bigram_counts += Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts += Counter(ngrams(string.split(), 3)).most_common(10)


In [188]:
# Create a list of common bigrams and trigrams from the custom corpus
common_ngrams = bigram_counts + trigram_counts
common_ngram_list = []
for each_ngram in common_ngrams:
    common_ngram_list.append("_".join(each_ngram[0]))

In [189]:
# Search for research keyword n-grams from the custom corpus and clean them
ngram_test_tokens = []

for text in test_tokens:
    cleaned_text = []
    for word in text:
        cleaned_text.append(clean_text(word))
    ngram_test_tokens.append(search_for_ngrams(unique_research_interest + common_ngram_list, cleaned_text))

In [3]:
# Create a word2vec model that contains all research keywords
research_tokens = convert_to_2d(research_interest_1d, research_interest_per_person)
research_word2vec = Word2Vec(research_tokens, size = len(unique_research_interest), min_count = 0)


In [4]:
# Train the word2vec model from the cleaned, n-grammed Wikipedia tokens
research_word2vec.build_vocab(ngram_wiki_tokens, update = True)
research_word2vec.train(ngram_wiki_tokens, total_examples = research_word2vec.corpus_count, epochs = research_word2vec.iter)

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(3722252, 4033045)

In [5]:
# Train the word2vec model by including the cleaned, n-grammed custom corpus tokens
research_word2vec.build_vocab(ngram_test_tokens, update = True)
research_word2vec.train(ngram_test_tokens, total_examples = research_word2vec.corpus_count, epochs = research_word2vec.iter)

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(148227, 162805)

In [6]:
# Example call to find similar analogies (e.g. computer_graphics + computer - graphics)
research_word2vec.wv.most_similar(positive=['computer_graphics', 'computer'], negative=['graphics'])

[('a_computer_network', 0.8285855054855347),
 ('computer_security', 0.828308641910553),
 ('embodying', 0.826147198677063),
 ('apollo', 0.825186550617218),
 ('computer_network', 0.8246166706085205),
 ('riskiest', 0.8225147128105164),
 ('detectedtoday', 0.8119497299194336),
 ('graphicstoday', 0.8097743988037109),
 ('human_computer', 0.8086000084877014),
 ('advocacy', 0.8070482015609741)]

In [7]:
# Example call to retrieve vector of a word
research_word2vec["ad_hoc_and_mobile_networks"]

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([-8.14489322e-04, -7.57868518e-04,  1.67428353e-03,  6.09227398e-04,
        1.45463180e-03, -5.45911083e-04, -1.08045142e-03, -7.04557635e-04,
        7.80204718e-04,  1.33535604e-03, -1.33158138e-03,  7.15423026e-04,
        1.70178781e-03, -1.10917681e-04, -8.67821334e-04, -3.33364180e-04,
        1.56960008e-03,  1.82358257e-03,  5.93353936e-04, -1.94017877e-04,
       -6.65026077e-04,  1.62628817e-03,  5.91923483e-04,  2.59567081e-04,
       -2.65831244e-04,  1.22209522e-03, -1.17180182e-03,  7.84935837e-04,
       -7.99558533e-04,  5.38808585e-04, -1.30341703e-03, -4.95403656e-04,
       -8.51318124e-04, -6.33128395e-04, -9.24593187e-04,  5.95878344e-04,
       -1.76619261e-03, -4.36472095e-04, -1.53125511e-04,  1.29962887e-03,
        1.82450691e-03,  6.81809091e-04,  3.15057288e-04, -1.05858047e-03,
        6.98117219e-05,  5.50869503e-04,  1.40601397e-03, -7.38345261e-04,
       -1.48160546e-03,  1.58775924e-03, -1.32333196e-03, -1.21970836e-03,
        1.21662905e-03, -

In [8]:
# Calculate similarity score between two keywords
research_word2vec.wv.similarity("computer_graphics", "graphics")

0.91753584

In [9]:
# Find the top 5 most similar keywords for all research keywords
for keyword in sorted(unique_research_interest):
    print("Top 5 similar words from "+keyword+": ")
    print(research_word2vec.wv.most_similar(keyword, topn = 5))
    print()

Top 5 similar words from ad_hoc_and_mobile_networks: 
[('erence', 0.26054614782333374), ('nucleosomes', 0.25874295830726624), ('prisons', 0.23829664289951324), ('autoregression', 0.23707222938537598), ('a_set', 0.22918835282325745)]

Top 5 similar words from agent_oriented_software_engineering: 
[('software_engineering', 0.9575194716453552), ('aose', 0.8987464308738708), ('timean', 0.8928987979888916), ('applicationdomain', 0.8880163431167603), ('gambit', 0.8872327208518982)]

Top 5 similar words from algorithms: 
[('techniques', 0.9139891862869263), ('methods', 0.8964818120002747), ('machine_learning_techniques', 0.8924683928489685), ('naturallanguageprocessing', 0.8909852504730225), ('nonai', 0.8866652250289917)]

Top 5 similar words from algorithms_and_data_structure: 
[('doublespending', 0.19441118836402893), ('mimofriendliness', 0.1740821897983551), ('segregated', 0.15225352346897125), ('solutions—they', 0.1514255851507187), ('cleans', 0.15121625363826752)]

Top 5 similar words fr

[('field_computer_networking', 0.9112381339073181), ('systemsin', 0.9088703989982605), ('disseminating', 0.8997477889060974), ('framegrabber', 0.8869712352752686), ('debatedmuch', 0.8845059275627136)]

Top 5 similar words from computer_supported_cooperative_work: 
[('ieeeacm', 0.9849088788032532), ('altran', 0.9808815717697144), ('died', 0.9804302453994751), ('courses', 0.9794739484786987), ('teach', 0.9788440465927124)]

Top 5 similar words from computer_vision: 
[('vision', 0.970188319683075), ('computer_science', 0.9515944123268127), ('discipline_computer_science', 0.9420118927955627), ('ascribed', 0.930199921131134), ('debatedmuch', 0.9230660200119019)]

Top 5 similar words from critical_information_infrastructure_protection: 
[('information_infrastructure', 0.9837023019790649), ('critical_information_infrastructure', 0.9836264252662659), ('critical_information', 0.9755337834358215), ('publish', 0.9703199863433838), ('stakeholders', 0.9691755175590515)]

Top 5 similar words from cr

[('“typically', 0.25708475708961487), ('novell', 0.23241958022117615), ('schematics', 0.2250405251979828), ('nonfriends', 0.2226688116788864), ('applicationoriented', 0.22260691225528717)]

Top 5 similar words from green_computing: 
[('definition_cloud_computing', 0.9788075685501099), ('reconfigurable_computing', 0.9733934998512268), ('grid_computing', 0.9733456373214722), ('00references', 0.9685972929000854), ('pervasive_computing', 0.9660100936889648)]

Top 5 similar words from grid_computing: 
[('grid', 0.983453631401062), ('green_computing', 0.9733458161354065), ('definition_cloud_computing', 0.9716693758964539), ('utility_computing', 0.9645079374313354), ('reconfigurable_computing', 0.9583411812782288)]

Top 5 similar words from hardware_accelerators: 
[('accelerators', 0.909809947013855), ('baseband', 0.8787966966629028), ('specialized', 0.871354341506958), ('generalpurpose', 0.8689441084861755), ('application_specific_processors', 0.8581291437149048)]

Top 5 similar words from h

[('analysis_tools', 0.9402003288269043), ('performance_analysis_tools', 0.9367772936820984), ('sentiment_analysis', 0.930321991443634), ('emphasize', 0.9302043914794922), ('innumeracy', 0.9284937381744385)]

Top 5 similar words from multimedia_annotation: 
[('codedin', 0.21778038144111633), ('the_blood_glucose', 0.21098965406417847), ('trophicfactors', 0.2049400359392166), ('uplink', 0.19209548830986023), ('interactive_sentient_environments', 0.19192826747894287)]

Top 5 similar words from multimedia_communication: 
[('of_multimedia_communication', 0.9659978747367859), ('of_multimedia', 0.9572462439537048), ('form_of_multimedia', 0.9528594613075256), ('a_form_of', 0.9438357353210449), ('improves', 0.9378102421760559)]

Top 5 similar words from multimedia_computing_and_networking: 
[('survey”', 0.22386617958545685), ('datatransfer', 0.19441847503185272), ('mpc', 0.1842622458934784), ('distributed_graph_computing', 0.18191595375537872), ('graphical_user_interface', 0.17858517169952393)]


[('leveraged', 0.9789721965789795), ('brought', 0.9781419038772583), ('applicability', 0.9777195453643799), ('groupware', 0.976702868938446), ('cutting_force', 0.9756389260292053)]

Top 5 similar words from semantic_transcoding_of_video_streams: 
[('predicated', 0.20709769427776337), ('highfunctionality', 0.20066815614700317), ('andnd', 0.18515479564666748), ('dspspecific', 0.18440845608711243), ('semiautonomously', 0.18380285799503326)]

Top 5 similar words from semantic_web: 
[('the_semantic_web', 0.9858883023262024), ('semantic', 0.9652156233787537), ('web_page', 0.9275813698768616), ('rdf', 0.922960102558136), ('retrieves', 0.9227731823921204)]

Top 5 similar words from sensor_networks: 
[('wireless_sensor_networks', 0.9966486692428589), ('wireless_sensor', 0.987119197845459), ('local_area_networks', 0.9845135807991028), ('area_networks', 0.9817748069763184), ('hoc_networks', 0.9794394969940186)]

Top 5 similar words from sentiment_analysis: 
[('sentiment', 0.9816864132881165), ('a

In [10]:
# One Hot Encoding for name, group and university
name_label_encoder = LabelEncoder()
uni_label_encoder = LabelEncoder()
group_label_encoder = LabelEncoder()
research_label_encoder = LabelEncoder()

onehot_encoder = OneHotEncoder(sparse=False)

name_integer_encoded = name_label_encoder.fit_transform(name)
name_integer_encoded = name_integer_encoded.reshape(len(name_integer_encoded), 1)
name_onehot = onehot_encoder.fit_transform(name_integer_encoded)

group_integer_encoded = group_label_encoder.fit_transform(group)
group_integer_encoded = group_integer_encoded.reshape(len(group_integer_encoded), 1)
group_onehot = onehot_encoder.fit_transform(group_integer_encoded)

university_integer_encoded = uni_label_encoder.fit_transform(array(university_1d))
university_integer_encoded = university_integer_encoded.reshape(len(university_integer_encoded), 1)
university_onehot = onehot_encoder.fit_transform(university_integer_encoded)


In [11]:
# Convert the numpy arrays to lists
name_onehotlist = name_onehot.tolist()
group_onehotlist = group_onehot.tolist()
university_onehotlist = university_onehot.tolist()



In [13]:
# Map each research keyword as a weight vector that measures its similarity with all other research keywords
research_interest_weight_dict = defaultdict(list)
for keyword1 in sorted(unique_research_interest):
    keyword1_weights = []
    for keyword2 in sorted(unique_research_interest):
        # Round to 5 decimal places for all similarity scores
        keyword1_weights.append(round(research_word2vec.wv.similarity(w1=keyword1, w2=keyword2), 5))
    research_interest_weight_dict[keyword1] = keyword1_weights

In [14]:
# Provide weight vectors for each research interest
research_interest_weights = []
for res_int in research_interest_1d:
    research_interest_weights.append(research_interest_weight_dict[res_int])
    

In [15]:
# Problem: each professor has multiple universities and research interests
# Convert these 1D arrays to 2D, to associate them to each professor

university_onehotlist_processed = []
research_interest_onehotlist_processed = []

university_onehotlist_processed = convert_to_2d(university_onehotlist, universities_per_person)
research_interest_processed = convert_to_2d(research_interest_weights, research_interest_per_person)

In [16]:
# Create an array that allows multiple universities and research interests per professor by summation
university_2d = []
for universities_per_professor in university_onehotlist_processed:
    university_2d.append([sum(x) for x in zip(*universities_per_professor)])
    
research_interest_2d = []
for research_interests_per_professor in research_interest_processed:
    research_interest_2d.append([sum(x) for x in zip(*research_interests_per_professor)])

# Since universities can be duplicate, change so that the maximum value of each university is 1
for universities_per_professor in university_2d:
    for index in range(0, len(universities_per_professor)):
        if universities_per_professor[index] > 1:
            universities_per_professor[index] = 1.0


In [17]:
# Normalise the values of all research keyword vectors to a range between 0 and 1
normalised_research_interest_2d = []
for research_vector in research_interest_2d:
    normalised_research_interest_2d.append(normalise_vector(research_vector))


In [18]:
# Example 2D research interest vector. This is the research keyword vector for Alexei Sourin
print(normalised_research_interest_2d[0])

[0.1980548993634697, 0.7939608740980841, 0.681849382362824, 0.17299590690812805, 0.17679453940083167, 0.6544278233914053, 0.4216865442417775, 0.7296511458079877, 0.10425806907525839, 0.9437780139014552, 0.8814956743503992, 0.16135748003829353, 0.6196188167219038, 0.6019963220073861, 0.8592352696824429, 0.09205829362670657, 0.9662591775785111, 0.18422013323374808, 0.5044226512442108, 0.15384468488866926, 0.8376969530484012, 0.19526451407651205, 0.6493538936893645, 0.5909247037595987, 0.9505932269412635, 0.9415462447605986, 0.1996953451391746, 0.926962101166148, 0.11093701425963506, 0.7947647399820457, 0.08379885211260167, 0.1824952155919855, 0.13289769821817224, 0.5290783446522942, 0.150070575767738, 0.13666363281839997, 0.1802743498514678, 0.9045300380349183, 0.7451644987091148, 0.09722214740254327, 0.13402584498205541, 0.06944088779781495, 0.17706431660273703, 0.9368919631789461, 0.7415375498677811, 0.9259211836035277, 0.9517731504386332, 0.3056783231640817, 0.8516543539922272, 0.9075

In [19]:
# Define the schema for the FusionART model
model_schema = [{'name': data.columns[0], 'compl': False, 'attrib': sorted(unique_name)}, 
                {'name': data.columns[1], 'compl': False, 'attrib': sorted(unique_group)},
                {'name': data.columns[2], 'compl': False, 'attrib': sorted(unique_university)},
                {'name': data.columns[3], 'compl': True, 'attrib': sorted(unique_research_interest)}]

In [20]:
# Initialise the FusionART model
model = FusionART(schema = model_schema, beta = [1.0, 1.0, 1.0, 1.0], alpha = [0.1, 0.1, 0.1, 0.1], 
               gamma = [0.25, 0.25, 0.25, 0.25], rho = [1, 1, 1, 1])
model.F1Fields

[{'name': 'name',
  'compl': False,
  'attrib': ['AS Madhukumar',
   'Alexei Sourin',
   'Anupam Chattopadhyay',
   'Anwitaman Datta',
   'Arijit Khan',
   'Arvind Easwaran',
   'Bo An',
   'Cai Jianfei',
   'Cai Wentong',
   'Cham Tat Jen',
   'Chan Syin',
   'Chia Liang Tien Clement',
   'Chng Eng Siong',
   'Deepu Rajan',
   'Deng Ruilong',
   'Douglas Leslie Maskell',
   'Dusit Niyato',
   'Eric Cambria',
   'Gao Cong',
   'Goh Wooi Boon',
   'Guan Cuntai',
   'He Ying',
   'Huang Shell Ying',
   'Hui Siu Cheung',
   'Jagath C. Rajapakse',
   'Ke Yiping Kelly',
   'Kong Wai Kin Adams',
   'Kwoh Chee Keong',
   'Lam Kwok Yan',
   'Lam Siew Kei',
   'Lau Chiew Tong',
   'Lee Bu Sung Francis',
   'Li Fang Flora',
   'Li Mo',
   'Li Yi',
   'Liang Qianhui Althea',
   'Lin Feng',
   'Lin Guosheng',
   'Lin Shang Wei',
   'Lin Weisi',
   'Liu Weichen',
   'Liu Yang',
   'Loke Yuan Ren',
   'Long Cheng',
   'Loy Chen Change (Cavan)',
   'Lu Shijian',
   'Luo Jun',
   'Mahardhika Pratama',

In [21]:
# Store the data to the Fusion ART model

for i in range(0, len(name)):
    model.updateF1bySchema([{'name': data.columns[0], 'val': name_onehotlist[i]}, 
                            {'name': data.columns[1], 'val': group_onehotlist[i]}, 
                            {'name': data.columns[2], 'val': university_2d[i]}, 
                            {'name': data.columns[3], 'val': normalised_research_interest_2d[i]}])

    
    print("resonance search: ")
    J = model.resSearch()
    print("selected ", J)
    if model.uncommitted(J):
        print ('uncommitted')

    model.autoLearn(J)
    model.doReadoutAllFields(J)
                    
    
model.displayNetwork()

resonance search: 
selected  0
uncommitted
resonance search: 
selected  1
uncommitted
resonance search: 
selected  2
uncommitted
resonance search: 
selected  3
uncommitted
resonance search: 
selected  4
uncommitted
resonance search: 
selected  5
uncommitted
resonance search: 
selected  6
uncommitted
resonance search: 
selected  7
uncommitted
resonance search: 
selected  8
uncommitted
resonance search: 
selected  9
uncommitted
resonance search: 
selected  10
uncommitted
resonance search: 
selected  11
uncommitted
resonance search: 
selected  12
uncommitted
resonance search: 
selected  13
uncommitted
resonance search: 
selected  14
uncommitted
resonance search: 
selected  15
uncommitted
resonance search: 
selected  16
uncommitted
resonance search: 
selected  17
uncommitted
resonance search: 
selected  18
uncommitted
resonance search: 
selected  19
uncommitted
resonance search: 
selected  20
uncommitted
resonance search: 
selected  21
uncommitted
resonance search: 
selected  22
uncommitte

Code: 73 {'F2': 0, 'weights': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.18073

In [22]:
# Perform the research keywords query here

def query_by_research_with_noise(query_research, output_file, noise_limit):
    
    query_research = query_research.split(";")
    output_file.write("Research Keywords: ")
    output_file.write(', '.join(query_research)+"\n")

    # Preprocess the research keyword queries to the format used for word embedding (phrases)
    for index in range(0, len(query_research)):
        query_research[index] = query_research[index].replace(" ", "_")
        query_research[index] = query_research[index].lower()
        query_research[index] = query_research[index].replace(":", "")
        query_research[index] = query_research[index].replace(",", "")

    # Convert all the research keywords into input vectors
    for research_index in range(0, len(query_research)):
        query_research[research_index] = research_interest_weight_dict[query_research[research_index]]


    # Remove keywords that could not be converted to input vectors
    index = 0
    while (index < len(query_research)):
        if type(query_research[index]) != list:
            del query_research[index]
        else:
            index += 1
        
    # Combine all the research keywords inputs into one vector that contains all research keywords
    research_vector = [sum(research_ints) for research_ints in zip(*query_research)]
    
    normalised_research_vector = normalise_vector(research_vector)
    

    # With a defined probability, flip the binary research keyword vectors due to noise
    for index in range(0, len(research_vector)):
        noise_value = random.random()
        if (noise_value < noise_limit):
            normalised_research_vector[index] = 1 - normalised_research_vector[index]
            
    # Define the complement-coded research vector
    compl_vector = []
    for bit in normalised_research_vector:
        compl_vector.append(1 - bit)
    

    model.setParam('gamma', [0,0,0,1])
    model.setParam('rho', [0,0,0,0])
    model.updateF1bySchema([{'name': data.columns[3], 'val': normalised_research_vector, 'vcompl': compl_vector}], refresh = False)
    # model.updateF1bySchema([{'name': data.columns[3], 'val': normalised_research_vector}])
    model.compChoice()

    # Find the nodes that have the highest F2 value
    maxF2value = model.codes[0]['F2']
    maxF2indexes = []

    for i in range(0, len(model.codes)-1):
        F2value = model.codes[i]['F2']
        if (F2value > maxF2value):
            maxF2value = F2value
            maxF2indexes = [i]
        elif (F2value == maxF2value):
            maxF2indexes.append(i)

    # Retrieve the data of the node(s) that have the highest F2 value
    output_file.write("Professor that follows these research keywords:")    
    for node in maxF2indexes:
        name_vector = model.codes[node]['weights'][0]
        for index in range(0, len(name_vector)):
            # Iterate through the vector and find the indexes that has a value of 1
            if (name_vector[index] == 1):
                # Retrieve the data from the model's 'attrib' attribute in F1 
                retrieved_name = model.F1Fields[0]['attrib'][index]
                output_file.write(retrieved_name)
    
    output_file.write("\n\n")
    return(retrieved_name)


In [24]:
# Research keywords query for 0% noise
output_file = open("output_zero_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()


In [24]:
# Research keywords query for 10% noise
output_file = open("output_ten_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.1)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [25]:
# Research keywords query for 20% noise
output_file = open("output_twenty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.2)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [26]:
# Research keywords query for 30% noise
output_file = open("output_thirty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.3)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [27]:
# Research keywords query for 40% noise
output_file = open("output_forty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.4)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [28]:
# Research keywords query for 50% noise
output_file = open("output_fifty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.5)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [29]:
# Research keywords query for 60% noise
output_file = open("output_sixty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.6)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [30]:
# Research keywords query for 70% noise
output_file = open("output_seventy_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.7)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [31]:
# Research keywords query for 80% noise
output_file = open("output_eighty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.8)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [32]:
# Research keywords query for 90% noise
output_file = open("output_ninety_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.9)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [33]:
# Research keywords query for 100% noise
output_file = open("output_hundred_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 1)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
